# Imports

In [47]:
import requests
from bs4 import BeautifulSoup
import sqlite3 as sql3
from datetime import datetime

# First step: Extract data using web scraping

In [37]:
# create a variable to store the link that will be scrapped
stock_target = "TSLA"
base_url = "https://finance.yahoo.com/quote/"

In [38]:
# get data from the page using requests library
raw_data = requests.get(base_url + stock_target)

# verify if the request worked: status 200 means success
print(raw_data.status_code)


200


In [43]:
# since it retrieves a lot of data, store it in a html file to analyze, makes it easier
with open('html_code.html', 'w') as f:
    f.write(raw_data.text)

# Transform

In [39]:
soup = BeautifulSoup(raw_data.text, 'html.parser')

In [45]:
# analyzying the raw data, the stock price is inside a <span> that is inside a <fin-streamer> tag
# the specific <fin-streamer> tag that holds the price has this specific attribute: data-test="qsp-price"
fin_streamers_tag_value = soup.find_all('fin-streamer', {'data-test': 'qsp-price'})

# storing the stock price in a variable
stock_price = fin_streamers_tag_value[0].text

# verifying if it worked
print(stock_price)

254.85


# Load

In [48]:
# I will store the data using the native SQLite database of Python, because it doesn't require any extra installation
conn = sql3.connect('stock_prices.db')
cursor = conn.cursor()

In [50]:
# create table if doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS stock_data (
        id INTEGER PRIMARY KEY,
        symbol TEXT NOT NULL,
        price REAL NOT NULL,
        timestamp DATETIME NOT NULL
    )
''')

In [54]:
# prepare the SQL command string to be insert the new value
# sql_string = f"INSERT INTO stock_data (symbol, price, timestamp) VALUES ({stock_target}, {stock_price}, {datetime.now()})"

# execute the command
cursor.execute('INSERT INTO stock_data (symbol, price, timestamp) VALUES (?, ?, ?)', (stock_target, stock_price, datetime.now()))

In [58]:
# retrieving the data to verify of it worked

data = cursor.execute('SELECT * FROM stock_data')
print([row for row in data])

[(1, 'TSLA', 254.85, '2023-10-17 17:45:07.646911')]
